## Máster en Ingeniería informática: Datos, Cloud y Gestión TI

# Análisis de información no estructurada

## Generador de letras de canciones

#### Realizado por: Arturo Pérez Sánchez y Jacinto Ruiz Díaz

---

El objetivo de este proyecto es el de implementar un sistema que permita generar de manera automática letras de canciones imitando el estilo de un artista en particular, realizando previamente una lectura de la letra de sus canciones.

## Recogida de datos: web scraping

El primer paso a realizar es la extracción de las letras de canciones para su posterior análisis. En nuestro caso, para obtener las letras de las canciones hemos optado por realizar <a href="https://es.wikipedia.org/wiki/Web_scraping">web scrapping</a> de la página <a href="https://www.azlyrics.com/">azlyrics</a>, la cual almacena letras de una amplia variedad de autores.

Para realizar la extracción de las letras utilizaremos la libreria <a href="https://www.crummy.com/software/BeautifulSoup/bs4/doc/">BeautifulSoup</a>, al ser una de las librerías de Python más completas e intuitivas para realizar este tipo de tareas.

Comenzaremos importándola:

In [1]:
import requests
from bs4 import BeautifulSoup
import time
import json
import io
import re

Una vez importada la librería definiremos algunos Hiperparámetros:
<ul>
    <li><strong>group:</strong> Nombre del grupo musical cuyo estilo deseamos imitar.</li>
    <li><strong>albums:</strong> Lista de álbumes del grupo que vamos a analizar (si se deja vacío se leerán todos).</li>
    <li><strong>wait_time:</strong> Tiempo en segundos que transcurrirá entre una petición y otra.</li>
</ul>

In [2]:
group = 'Lady Gaga'
albums = ['Red And Blue', 'The Fame', 'The Fame Monster', 'Born This Way', 'A Very Gaga Holiday', 'Artpop', 'Joanne', 'Chromatica']
wait_time = 8 # Azlyrics baneó varias veces nuestras IPs por hacer consultas continuadas a su web. 
              # Hemos calculado que un tiempo prudencial entre consultas es de 8 segundos 

Lo primero que tenemos que realizar es una conversión del nombre del grupo al nombre utilizado en la página (todo en minúsculas y sin espacios):

In [3]:
group = group.lower().replace(" ", "")

Ahora procedemos a estudiar cómo se estructuran las URLs del sitio web al que vamos a hacer scrapping.

Concretamente, estamos interesados en __dos__ vistas del sitio:

### Lista de canciones separadas por álbumes
La URL de esta página se estructura de la forma:
<h3><center>https://www.azlyrics.com/<font color="red">[initial]</font>/<font color="red">[group]</font>.html </center></h3>
Donde <i>[initial]</i> corresponde a la primera letra del grupo* y <i>[group]</i> corresponde al nombre del grupo. En el ejemplo del grupo Lady Gaga, se vería de la siguiente forma:

<img src="images/gaga_album_example.png" width=600>
<br>

*_Si el nombre del grupo comienza por un carácter especial la variable [initial] tomará el valor '19'_

### Letra de cada canción
En este caso, notaremos que la ruta es la misma pero cambiando [initial] por 'lyrics' y añadiendo el nombre de la canción justo después del nombre del grupo:

<h3><center> https://www.azlyrics.com/lyrics/<font color="red">[group]</font>/<font color="red">[song]</font>.html </center></h3>

El resultado para el ejemplo de la canción "Poker Face" de Lady Gaga es el siguiente:

<img src="images/gaga_lyrics2.png" width=600>

Con esto, ya podemos estructurar la ruta de álbumes a partir de los hiperparámetros que habíamos establecido:

In [5]:
#Inicializamos la variable initial como el primer caracter del grupo o 19 si no es una letra (perteneciente al #)
initial = group[0] if group[0].isalpha() else '19'

#Prefijo del sitio web
prefix = 'https://www.azlyrics.com'
sufix = '.html'

#Concatenamos las variables para formar la URL completa
albumsUrl = prefix + '/' + initial + '/' + group

print(albumsUrl)

https://www.azlyrics.com/l/ladygaga


Con la URL del artista ya creado (donde se almacenan sus álbumes), podemos utilizar las librerías anteriormente mencionadas para hacer la llamada a la página y obtener la lista de álbumes y los títulos de las canciones que lo componen:

In [6]:
# Las claves serán los álbumes y los valores un array con las canciones. Las canciones a su vez serán un array de tamaño 3:
# El primer valor es el titulo, el segundo el enlace y el tercero la letra de la canción
albumsDict = {}

# Hacemos la llamada para obtener la lista de álbumes
r = requests.get(albumsUrl + sufix)
r.encoding = 'utf-8' #Codificamos el request a utf-8

soup = BeautifulSoup(r.text, 'lxml')

res = soup.findAll('div', class_=['album', 'listalbum-item'])

# Recorremos la lista de álbumes y para cada uno de ellos guardamos en el diccionario
# el título de cada canción y la ruta a la letra de dicha canción
currentAlbum = ''
for div in res:
    if(div.get('class')[0] == 'album'):
        # Este div corresponde a un álbum, por lo que añadimos una nueva entrada al diccionario
        currentAlbum = div.find('b').contents[0].replace('"', '')
        albumsDict[currentAlbum] = []
    else:
        # Este div corresponde a una canción por lo que la añadimos a los valores del álbum
        # Hay veces que referencia a una ruta completa, al pertenecer a otro artista, por lo que hay que tenerlo en cuenta
        if div.contents[0]['href'][2:].startswith("tps://www.azlyrics.com"):
            albumsDict[currentAlbum].append([div.contents[0].contents[0], div.contents[0]['href']])
        else:
            albumsDict[currentAlbum].append([div.contents[0].contents[0], 'https://www.azlyrics.com' + div.contents[0]['href'][2:]])

Vemos el aspecto de este albumsDict:

In [9]:
albumsDict

{'Red And Blue': [['Something Crazy',
   'https://www.azlyrics.com/lyrics/ladygaga/somethingcrazy.html'],
  ['Wish You Were Here',
   'https://www.azlyrics.com/lyrics/ladygaga/wishyouwerehere.html'],
  ['No Floods', 'https://www.azlyrics.com/lyrics/ladygaga/nofloods.html'],
  ['Red And Blue', 'https://www.azlyrics.com/lyrics/ladygaga/redandblue.html'],
  ['Words', 'https://www.azlyrics.com/lyrics/ladygaga/words.html']],
 'The Fame': [['Just Dance',
   'https://www.azlyrics.com/lyrics/ladygaga/justdance.html'],
  ['LoveGame', 'https://www.azlyrics.com/lyrics/ladygaga/lovegame.html'],
  ['Paparazzi', 'https://www.azlyrics.com/lyrics/ladygaga/paparazzi.html'],
  ['Beautiful, Dirty, Rich',
   'https://www.azlyrics.com/lyrics/ladygaga/beautifuldirtyrich.html'],
  ['Eh, Eh (Nothing Else I Can Say)',
   'https://www.azlyrics.com/lyrics/ladygaga/ehehnothingelseicansay.html'],
  ['Poker Face', 'https://www.azlyrics.com/lyrics/ladygaga/pokerface.html'],
  ['The Fame', 'https://www.azlyrics.com/l

Ahora tenemos un diccionario con todos los álbumes, y para cada álbum tenemos una lista con todas las canciones que lo componen, con sus respectivos enlaces a la letra. Por tanto, solo nos faltaría llamar a cada uno de estos enlaces para obtener la letra de cada canción; sin embargo, para reducir el número de llamadas al sitio web, primero realizaremos una limpieza de los álbumes para quedarnos solo con aquellos que habíamos indicado que nos interesaban:

In [7]:
# Limpieza de álbumes
# Del diccionario que hemos creado nos quedamos solo con los álbumes 
# que aparezcan en el array de álbumes que habíamos especificado inicialmente o con todos si no se ha especificado
if(albums):
    albumsDict = {a: albumsDict[a] for a in albums if a in albumsDict}

Ahora sí que tenemos todo listo para realizar la lectura de las canciones. Cabe destacar que entre cada llamada dejaremos un espacio de tiempo (indicado en la variable '_wait_time_') para evitar que el sitio web nos bloquee la dirección IP. Para conseguirlo haremos uso de la librería **time**.

In [8]:
#Creamos un diccionario donde las claves serán las canciones y el valor la letra de dicha canción
lyricsDict = {}

print('Leyendo ' + str(len(albumsDict)) + ' álbum(es)')

#Recorremos la lista de álbumes y para cada álbum recorremos la lista de canciones
for album in albumsDict:
    print ('Reading album: ', album)
    for song in albumsDict[album]:
        print ('    Reading song: ', song[0])
        
        # Esperamos unos segundos antes de realizar la llamada
        time.sleep(wait_time)
        
        r = requests.get(song[1])
        r.encoding = 'utf-8' #Codificamos el request a utf-8
        
        soup = BeautifulSoup(r.text, 'lxml')
        
        # Como el div que contiene la letra no tiene ninguna clase ni identificador lo obtendremos a partir del div padre
        column = soup.find('div', class_=['col-xs-12 col-lg-8 text-center'])
        
        #El div con la letra de la canción siempre estará en 5º lugar, después de los divs del título y de las redes sociales
        raw_lyrics = column.findAll('div')[5].text
        raw_lyrics = re.sub("[\(\[].*?[\)\]]", "", raw_lyrics) #Quitamos el contenido entre paréntesis y corchetes
        lyricsDict[song[0]] = raw_lyrics

Leyendo 8 álbum(es)
Reading album:  Red And Blue
    Reading song:  Something Crazy
    Reading song:  Wish You Were Here
    Reading song:  No Floods
    Reading song:  Red And Blue
    Reading song:  Words
Reading album:  The Fame
    Reading song:  Just Dance
    Reading song:  LoveGame
    Reading song:  Paparazzi
    Reading song:  Beautiful, Dirty, Rich
    Reading song:  Eh, Eh (Nothing Else I Can Say)
    Reading song:  Poker Face
    Reading song:  The Fame
    Reading song:  Money Honey
    Reading song:  Again Again
    Reading song:  Boys Boys Boys
    Reading song:  Brown Eyes
    Reading song:  Summerboy
    Reading song:  I Like It Rough
    Reading song:  Retro Dance Freak
Reading album:  The Fame Monster
    Reading song:  Bad Romance
    Reading song:  Alejandro
    Reading song:  Monster
    Reading song:  Speechless
    Reading song:  Dance In The Dark
    Reading song:  Telephone
    Reading song:  So Happy I Could Die
    Reading song:  Teeth
Reading album:  Born 

Vemos el aspecto del lyricsDict generado (diccionario con título de canción como clave y letra de la misma como valor):

In [10]:
lyricsDict

{'Something Crazy': "\n\r\nWait, I think it's time \nYour fire's lit and so is mine \nGo turn out the light \nDon't be afraid, tonight's the night \n\nAnd I won't, I won't try to change you \nBut I will, I will if I want to \nYeah, it always brings me down \nWhen you're not around \nI still need you \n\nOh, You do something for me baby \nI cannot control it, baby \nYou do something for me \nOh, You do something crazy to me \nI cannot control it, baby \nYou do something for me \n\nAnd I feel so down \nWhen you're not around \n\nBoy, we're almost there \nI'll lay your clothes over my chair \nPull the shades down low \nWe're both ready so lose control \n\nAnd I won't, I won't try to tame you \nBut I will, I will if you want to \nYeah it always brings me down \nWhen you're not around \nI still need you \n\nOh, You do something for me baby \nI cannot control it, baby \nYou do something for me \nOh, You do something crazy to me \nI cannot control it, baby \nYou do something for me \n\nAnd I 

Por último, tenemos que fusionar los arrays albumsDict y lyricsDict:

In [11]:
# Recorremos los álbumes
for album in albumsDict:
    # Recorremos las canciones de cada álbum
    for song in albumsDict[album]:
        #A cada canción le añadimos la letra correspondiente
        song.append(lyricsDict[song[0]])

Ahora ya tenemos todas las canciones tal y como queríamos, por lo que podemos guardarlas en un json para su posterior análisis:

In [12]:
# El json se guardará en la carpeta data
with open('data/' + group + "-lyrics.json", "w", encoding='utf-8') as outfile: 
    json.dump(albumsDict, outfile)

Vamos a abrir el fichero CSV generado para ver qué forma tiene:

In [13]:
with open('data/' + group + '-lyrics.json') as f:
     data = json.load(f)

print(json.dumps(data, indent=4))

{
    "Red And Blue": [
        [
            "Something Crazy",
            "https://www.azlyrics.com/lyrics/ladygaga/somethingcrazy.html",
            "\n\r\nWait, I think it's time \nYour fire's lit and so is mine \nGo turn out the light \nDon't be afraid, tonight's the night \n\nAnd I won't, I won't try to change you \nBut I will, I will if I want to \nYeah, it always brings me down \nWhen you're not around \nI still need you \n\nOh, You do something for me baby \nI cannot control it, baby \nYou do something for me \nOh, You do something crazy to me \nI cannot control it, baby \nYou do something for me \n\nAnd I feel so down \nWhen you're not around \n\nBoy, we're almost there \nI'll lay your clothes over my chair \nPull the shades down low \nWe're both ready so lose control \n\nAnd I won't, I won't try to tame you \nBut I will, I will if you want to \nYeah it always brings me down \nWhen you're not around \nI still need you \n\nOh, You do something for me baby \nI cannot control i

Sigue la forma: <br> <font size="3"> <pre>
{ nombreAlbum: [
    [
     tituloCancion, 
     urlCancion, 
     letraCancion
    ],
    ...
  ],
  ...
}
</pre></font>

Para mostrar un álbum en concreto:

In [14]:
#Álbum Red And Blue
data['Red And Blue']

[['Something Crazy',
  'https://www.azlyrics.com/lyrics/ladygaga/somethingcrazy.html',
  "\n\r\nWait, I think it's time \nYour fire's lit and so is mine \nGo turn out the light \nDon't be afraid, tonight's the night \n\nAnd I won't, I won't try to change you \nBut I will, I will if I want to \nYeah, it always brings me down \nWhen you're not around \nI still need you \n\nOh, You do something for me baby \nI cannot control it, baby \nYou do something for me \nOh, You do something crazy to me \nI cannot control it, baby \nYou do something for me \n\nAnd I feel so down \nWhen you're not around \n\nBoy, we're almost there \nI'll lay your clothes over my chair \nPull the shades down low \nWe're both ready so lose control \n\nAnd I won't, I won't try to tame you \nBut I will, I will if you want to \nYeah it always brings me down \nWhen you're not around \nI still need you \n\nOh, You do something for me baby \nI cannot control it, baby \nYou do something for me \nOh, You do something crazy t

Para mostrar una canción de un álbum en particular:

In [15]:
#Álbum Red And Blue, canción Something Crazy
data['Red And Blue'][0]

['Something Crazy',
 'https://www.azlyrics.com/lyrics/ladygaga/somethingcrazy.html',
 "\n\r\nWait, I think it's time \nYour fire's lit and so is mine \nGo turn out the light \nDon't be afraid, tonight's the night \n\nAnd I won't, I won't try to change you \nBut I will, I will if I want to \nYeah, it always brings me down \nWhen you're not around \nI still need you \n\nOh, You do something for me baby \nI cannot control it, baby \nYou do something for me \nOh, You do something crazy to me \nI cannot control it, baby \nYou do something for me \n\nAnd I feel so down \nWhen you're not around \n\nBoy, we're almost there \nI'll lay your clothes over my chair \nPull the shades down low \nWe're both ready so lose control \n\nAnd I won't, I won't try to tame you \nBut I will, I will if you want to \nYeah it always brings me down \nWhen you're not around \nI still need you \n\nOh, You do something for me baby \nI cannot control it, baby \nYou do something for me \nOh, You do something crazy to m

Para quedarnos con la letra de dicha canción:

In [16]:
#Álbum Red And Blue, letra de la canción Something Crazy
data['Red And Blue'][0][2]

"\n\r\nWait, I think it's time \nYour fire's lit and so is mine \nGo turn out the light \nDon't be afraid, tonight's the night \n\nAnd I won't, I won't try to change you \nBut I will, I will if I want to \nYeah, it always brings me down \nWhen you're not around \nI still need you \n\nOh, You do something for me baby \nI cannot control it, baby \nYou do something for me \nOh, You do something crazy to me \nI cannot control it, baby \nYou do something for me \n\nAnd I feel so down \nWhen you're not around \n\nBoy, we're almost there \nI'll lay your clothes over my chair \nPull the shades down low \nWe're both ready so lose control \n\nAnd I won't, I won't try to tame you \nBut I will, I will if you want to \nYeah it always brings me down \nWhen you're not around \nI still need you \n\nOh, You do something for me baby \nI cannot control it, baby \nYou do something for me \nOh, You do something crazy to me \nI cannot control it, baby \nYou do something for me \n\nAnd I feel so down \nWhen 

Para ver los nombres de los álbumes almacenados:

In [17]:
for i in data:
    print(i)

Red And Blue
The Fame
The Fame Monster
Born This Way
A Very Gaga Holiday
Artpop
Joanne
Chromatica


Para ver los títulos de las canciones de un álbum en particular:

In [18]:
for i in data['Red And Blue']:
    print(i[0])

Something Crazy
Wish You Were Here
No Floods
Red And Blue
Words


---

### Obteniendo todos los artistas disponibles de azlyrics

Esto lo estaremos usando en el despliegue de la aplicación en Django, para poder buscar entre los artistas de una manera rápida en la base de datos.

In [19]:
# Cada uno de los índices corresponde a una página del tipo https://www.azlyrics.com/{index}.html, que contiene los 
# artistas que empiezan por dicho carácter
indexes = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v',
           'w', 'x', 'y', 'z', '19']
baseUrl = "https://www.azlyrics.com/"
sufix = ".html"

wait_time = 8

for i in indexes:
    # Esperamos unos segundos antes de realizar la llamada
    time.sleep(wait_time)
    
    group = baseUrl + i + sufix
    
    r = requests.get(group)
    soup = BeautifulSoup(r.text, 'lxml')

    divs = soup.find_all('div', class_=['col-sm-6 text-center artist-col'])
    
    links = []
    for d in divs:
        links.extend(d.find_all('a'))
    
    for a in links:
        print(a.text + " " + baseUrl + a['href'])
    break # Como muestra, cortaremos el for en la primera iteración, quedándonos solo con los artistas que empiezan por A

A1 https://www.azlyrics.com/a/a1.html
A1 https://www.azlyrics.com/f/floyda1bentley.html
A1 x J1 https://www.azlyrics.com/a/a1xj1.html
A https://www.azlyrics.com/a/a.html
A2H https://www.azlyrics.com/a/a2h.html
A92 https://www.azlyrics.com/a/a92.html
Aalegra, Snoh https://www.azlyrics.com/s/snohaalegra.html
Aaliyah https://www.azlyrics.com/a/aaliyah.html
Aalto, Saara https://www.azlyrics.com/s/saaraaalto.html
Aaradhna https://www.azlyrics.com/a/aaradhna.html
Aaron Carpenter https://www.azlyrics.com/a/aaroncarpenter.html
Aaron Carter https://www.azlyrics.com/c/carter.html
Aaron Cole https://www.azlyrics.com/a/aaroncole.html
Aaron Doh https://www.azlyrics.com/a/aarondoh.html
Aaron Fresh https://www.azlyrics.com/a/aaronfresh.html
Aaron Goodvin https://www.azlyrics.com/a/aarongoodvin.html
Aaron Hall https://www.azlyrics.com/a/aaronhall.html
Aaron Lewis https://www.azlyrics.com/a/aaronlewis.html
Aaron Lines https://www.azlyrics.com/l/lines.html
Aaron May https://www.azlyrics.com/a/aaronmay.h

### Generando el fichero de letras

Ahora que tenemos de forma estructurada toda la información necesaria, recorreremos el json para almacenar en un fichero txt todas las letras de las canciones recogidas, que nos servirá para realizar el posterior entrenamiento del modelo.

In [20]:
file = io.open("song_lyrics.txt", "w", encoding="utf-8")  #Creación del fichero con nombre "song_lyrics.txt"

n_songs = 0

for album in data:
    for song in data[album]:
        file.write(song[2]) #El índice 2 contiene las letras
        n_songs += 1

print("Número total de canciones almacenadas: {}.\n".format(n_songs))

file.close()

print("song_lyrics.txt creado satisfactoriamente.")


Número total de canciones almacenadas: 94.

song_lyrics.txt creado satisfactoriamente.


Usaremos el fichero song_lyrics.txt para entrenar el modelo.

----

## Entrenamiento del modelo

Hacemos uso de la librería aitextgen.

In [1]:
#Importamos las librerías necesarias
# pip3 install aitextgen
from aitextgen.TokenDataset import TokenDataset
from aitextgen.tokenizers import train_tokenizer
from aitextgen.utils import GPT2ConfigCPU
from aitextgen import aitextgen
import torch

Verificamos en torch que tenemos CUDA instalado:

In [2]:
print(torch.cuda.current_device())
print(torch.cuda.get_device_name(torch.cuda.current_device()))

0
NVIDIA GeForce GTX 1660


In [3]:
# Indicamos el fichero a utilizar para entrenar el modelo
file_name = "song_lyrics.txt"

In [4]:
train_tokenizer(file_name)
tokenizer_file = "aitextgen.tokenizer.json"

In [5]:
# GPT2ConfigCPU is a mini variant of GPT-2 optimized for CPU-training
# e.g. the # of input tokens here is 64 vs. 1024 for base GPT-2.
config = GPT2ConfigCPU()

<b>Importante</b>: si no se tiene instalado CUDA, eliminar el parámetro "to_gpu=True" en la función aitextgen:

In [6]:
# Instantiate aitextgen using the created tokenizer and config. 
# Podría pasarse como atributo model='datificate/gpt2-small-spanish' para que el modelo base fuese un gpt2 en español, 
# pero no ha dado buenos resultados, por lo que por defecto se usa el de 124M de GPT-2 (inglés)
ai = aitextgen(tokenizer_file=tokenizer_file, config=config, to_gpu=True)

# You can build datasets for training by creating TokenDatasets,
# which automatically processes the dataset with the appropriate size.
data = TokenDataset(file_name, tokenizer_file=tokenizer_file, block_size=64)

In [26]:
# Train the model! It will save pytorch_model.bin periodically and after completion to the `trained_model` folder
ai.train(data, batch_size=16, num_steps=30000)

pytorch_model.bin already exists in /trained_model and will be overwritten!
Windows does not support multi-GPU training. Setting to 1 GPU.
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


1,000 steps reached: saving model to /trained_model
1,000 steps reached: generating sample texts.
 on your subway, red wt
R your cha-chase cle, burder play,
Scle, and hump drum, play, play da da da
Dum dum da da da da da da da da da da

Dum
2,000 steps reached: saving model to /trained_model
2,000 steps reached: generating sample texts.


Let's our eyes get lost you can take me home
Lifve up, pop out
'Cause I've been hurve
'Cause I'm a man
I'm beautiful in my way
'Cause God makes no mistakes no mistakes
3,000 steps reached: saving model to /trained_model
3,000 steps reached: generating sample texts.
, I'm beautiful in my way
'Cause God makes no mistakes, baby
I'm on the right track, baby
I was born this way
Don't hide yourself in regret
Just love yourself and you're set
I'm on the
4,000 steps reached: saving model to /trained_model
4,000 steps reached: generating sample texts.
, I was born this way

I'm on the right track, baby
I was born this way

Ooo there ain't no other way
Baby I w

28,000 steps reached: saving model to /trained_model
28,000 steps reached: generating sample texts.



I live for the fame fame baby 
The fame 
Isn't it a shame shame shame baby 
A shame 
A shame 
In it for the fame fame fame baby 
The fame fame 
Isn't it for it a shame shame baby 

29,000 steps reached: saving model to /trained_model
29,000 steps reached: generating sample texts.

Baby you're go
Introd fought to place

Come to mountil you love


'Cause I wanna be birp in the club
And I'm sipping that shitchxed

But I love you're not
30,000 steps reached: saving model to /trained_model
30,000 steps reached: generating sample texts.
 to me now

I on top of the world in my Fashion!
Look at me now!
I feel on top of the world in my Fashion!
Look at me on top of the world in my body
Looking good and feelin' serial kiss me s



Generamos ahora un texto con los siguientes parámetros:

In [62]:
ai.generate(n=1, max_length=300, temperature=1.2)

with right now 

'Cause I ever wanted was love
Hey yeah yeah 

All I ever wanted was love
Hey yeah yeah 
Hey yeah yeah 
Hey yeah, hey yeah yeah, hey yeah 

I want your stupid love
I want your stupid love, love


Lady Gaga tiene canciones cuyas letras no están muy bien formadas (expresiones coloquiales, palabras mal expresadas, letras en otros idiomas...), por lo que no parece generar buenos resultados. En otros grupos, como Birdy visto en la demo de la diapositiva de la entrega, es más consistente y menos problemático.